**Table of contents**<a id='toc0_'></a>    
- [CubeSharp by examples](#toc1_)    
  - [Helper Functions](#toc1_1_)    
    - [Helper functions for visualization](#toc1_1_1_)    
    - [Helper function to transform cube result into table](#toc1_1_2_)    
  - [Sample Data](#toc1_2_)    
  - [One-dimensional report (total quantity per customer)](#toc1_3_)    
  - [Two-dimensional report (quantity per year per customer)](#toc1_4_)    
  - [Two-dimensional report (quantity per tag per year; multi-selection dimension)](#toc1_5_)    
  - [Three-dimensional report (EmployeeId x CustomerId x Year)](#toc1_6_)    
  - [Zero / simple aggregation examples](#toc1_7_)    
  - [Modulus dimension examples](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[CubeSharp by examples](#toc0_)

In [24]:
// Dependencies
#r "nuget: CubeSharp, 99.0.0.3-alpha"

using CubeSharp;

Installed Packages CubeSharp, 99.0.0.3-alpha

## <a id='toc1_1_'></a>[Helper Functions](#toc0_)

### <a id='toc1_1_1_'></a>[Helper functions for visualization](#toc0_)

In [25]:
#r "nuget: Microsoft.Data.Analysis, 0.21.0"
#nullable enable

using System;
using System.Collections.Generic;
using System.Linq;
using System.Reflection;
using Microsoft.Data.Analysis;

public static void Display(this object? obj) => display(obj);

public static void DisplayAsTable(this IEnumerable<IDictionary<string, object?>> rows) =>
    display(rows.ToDataFrame());

public static IDictionary<string, object?> ObjectToDictionary(this object obj) =>
    obj.GetType()
        .GetProperties(BindingFlags.Public | BindingFlags.Instance)
        .ToDictionary(prop => prop.Name, prop => prop.GetValue(obj, null));

private static DataFrame ToDataFrame(this IEnumerable<IDictionary<string, object?>> source)
{
    var props = source.FirstOrDefault()?.Keys ?? [];
    var columns = props
        .Select(p => new StringDataFrameColumn(p))
        .ToList();
    var result = new DataFrame(columns);
    foreach (var item in source)
    {
        result.Append(props.ToDictionary(p => p, p => GetDisplayedValue(item[p])), true);
    }
    return result;

    static object? GetDisplayedValue(object? value) =>
        value switch
        {
            IEnumerable<object> values => $"[{values.JoinStrings(", ")}]",
            _ => value
        };
}

private static string JoinStrings(this IEnumerable<object?> strings, string separator) =>
    string.Join(separator, strings);


Installed Packages Microsoft.Data.Analysis, 0.21.0

### <a id='toc1_1_2_'></a>[Helper function to transform cube result into table](#toc0_)

In [ ]:
#nullable enable

public static IEnumerable<IDictionary<string, object?>> ToTable<TIndex, T>(
    this CubeResult<TIndex, T> cubeResult,
    Index[] rowDimensionNumbers,
    Index[] columnDimensionNumbers)
    where TIndex : notnull
{
    if (rowDimensionNumbers
        .Concat(columnDimensionNumbers)
        .GroupBy(x => x)
        .Any(g => g.Count() > 1))
        {
            throw new InvalidOperationException("Duplicated indexes are not allowed");
        }

    var shiftedColumnIndexes =
        ShiftIndexes(columnDimensionNumbers, rowDimensionNumbers, cubeResult.FreeDimensionCount);
    return cubeResult
        .BreakdownByDimensions(rowDimensionNumbers)
        .Select(row => row
            .GetBoundDimensionsAndIndexes()
            .Select(item => KeyValuePair.Create(
                item.dimension.Title ?? string.Empty,
                (object?)item.dimension[item.index].Title ?? item.index))
            .Concat(row
                .BreakdownByDimensions(shiftedColumnIndexes)
                .Select(column => KeyValuePair.Create(
                    column
                        .GetBoundDimensionsAndIndexes()[^columnDimensionNumbers.Length ..]
                        .Select(item => item.dimension[item.index].Title ?? item.index?.ToString())
                        .DefaultIfEmpty("value")
                        .JoinStrings("."),
                    (object?)column.GetValue())))
            .ToDictionary(kvp => kvp.Key, kvp => kvp.Value));
}

private static Index[] ShiftIndexes(Index[] indexes, Index[] precedingIndexes, int length) =>
    indexes
        .Select(index => index.GetOffset(length))
        .Select(index =>
            index - precedingIndexes.Count(i => i.GetOffset(length) < index))
        .Select(dimensionNumber => (Index)dimensionNumber)
        .ToArray();


## <a id='toc1_2_'></a>[Sample Data](#toc0_)

In [27]:
var orders = new [] {
    new { OrderDate = new DateTime(2007, 08, 02), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 10m, Tags = new [] { "Discount", "Retail" } },
    new { OrderDate = new DateTime(2007, 12, 24), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 12m, Tags = new [] { "Retail", "BestSeller" } },
    new { OrderDate = new DateTime(2007, 12, 24), Product = "Y", EmployeeId = 1, CustomerId = "B", Quantity = 20m, Tags = new [] { "Discount", "Retail", "New", "Season", "BestSeller" } },
    new { OrderDate = new DateTime(2008, 01, 09), Product = "Z", EmployeeId = 2, CustomerId = "A", Quantity = 40m, Tags = new [] { "BestSeller" } },
    new { OrderDate = new DateTime(2008, 01, 18), Product = "Z", EmployeeId = 1, CustomerId = "C", Quantity = 14m, Tags = new [] { "Discount", "New", "BestSeller" } },
    new { OrderDate = new DateTime(2008, 02, 12), Product = "Z", EmployeeId = 2, CustomerId = "B", Quantity = 12m, Tags = new [] { "Retail" } },
    new { OrderDate = new DateTime(2009, 02, 12), Product = "X", EmployeeId = 3, CustomerId = "A", Quantity = 10m, Tags = new string[] {} },
    new { OrderDate = new DateTime(2009, 02, 16), Product = "X", EmployeeId = 1, CustomerId = "C", Quantity = 20m, Tags = new [] { "New" } },
    new { OrderDate = new DateTime(2009, 04, 18), Product = "Z", EmployeeId = 2, CustomerId = "B", Quantity = 15m, Tags = new [] { "Discount", "BestSeller" } },
    new { OrderDate = new DateTime(2007, 04, 18), Product = "X", EmployeeId = 3, CustomerId = "C", Quantity = 22m, Tags = new [] { "Discount", "Retail", "New", "Season" } },
    new { OrderDate = new DateTime(2009, 09, 07), Product = "Y", EmployeeId = 3, CustomerId = "D", Quantity = 30m, Tags = new [] { "Retail", "New", "Season", "BestSeller" } },
    new { OrderDate = new DateTime(2009, 09, 07), Product = (string)null, EmployeeId = 3, CustomerId = "D", Quantity = 30m, Tags = new [] { "Discount", "New", "BestSeller" } },
};

orders
    .Select(order => order.ObjectToDictionary())
    .DisplayAsTable();

index,OrderDate,Product,EmployeeId,CustomerId,Quantity,Tags
0,8/2/2007 12:00:00 AM,X,3,A,10,"[Discount, Retail]"
1,12/24/2007 12:00:00 AM,X,3,A,12,"[Retail, BestSeller]"
2,12/24/2007 12:00:00 AM,Y,1,B,20,"[Discount, Retail, New, Season, BestSeller]"
3,1/9/2008 12:00:00 AM,Z,2,A,40,[BestSeller]
4,1/18/2008 12:00:00 AM,Z,1,C,14,"[Discount, New, BestSeller]"
5,2/12/2008 12:00:00 AM,Z,2,B,12,[Retail]
6,2/12/2009 12:00:00 AM,X,3,A,10,[]
7,2/16/2009 12:00:00 AM,X,1,C,20,[New]
8,4/18/2009 12:00:00 AM,Z,2,B,15,"[Discount, BestSeller]"
9,4/18/2007 12:00:00 AM,X,3,C,22,"[Discount, Retail, New, Season]"


## <a id='toc1_3_'></a>[One-dimensional report (total quantity per customer)](#toc0_)

In [28]:
var cube1d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(
        orders,
        o => o.Quantity,
        (a,b) => a + b,
        0m),
    DimensionDefinition.CreateForCollection(
        orders,
        o => o.CustomerId,
        title: null,
        IndexDefinition.Create("A"),
        IndexDefinition.Create("B"),
        IndexDefinition.Create("C"),
        IndexDefinition.Create("D")));

new {
    A = cube1d.GetValue("A"),
    B = cube1d.GetValue("B"),
    C = cube1d.GetValue("C"),
    D = cube1d.GetValue("D"),
}.Display();

A,72
B,47
C,56
D,60


## <a id='toc1_4_'></a>[Two-dimensional report (quantity per year per customer)](#toc0_)

In [29]:
var cube2d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.CustomerId, null,
        IndexDefinition.Create("A"), IndexDefinition.Create("B"), IndexDefinition.Create("C"), IndexDefinition.Create("D"))
            .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(orders, o => o.OrderDate.Year.ToString(), null,
        IndexDefinition.Create("2007"), IndexDefinition.Create("2008"), IndexDefinition.Create("2009")));

var tableCustomersYears = cube2d.BreakdownByDimensions(0)
    .Select(row => new {
        Customer = row.GetBoundIndexDefinition(0).Title ?? row.GetBoundIndexDefinition(0).Value,
        Y2007 = row.GetValue("2007"),
        Y2008 = row.GetValue("2008"),
        Y2009 = row.GetValue("2009"),
        Total = row.GetValue((string)default!),
    })
    .Select(order => order.ObjectToDictionary())
    .ToList();

tableCustomersYears.DisplayAsTable();

index,Customer,Y2007,Y2008,Y2009,Total
0,A,22,40,10,72
1,B,20,12,15,47
2,C,22,14,20,56
3,D,0,0,60,60
4,Total,64,66,105,235


## <a id='toc1_5_'></a>[Two-dimensional report (quantity per tag per year; multi-selection dimension)](#toc0_)

In [30]:
var cubeTagYear = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(orders, o => o.OrderDate.Year.ToString(), "Year",
        IndexDefinition.Create("2007"), IndexDefinition.Create("2008"), IndexDefinition.Create("2009"))
        .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollectionWithMultiSelector(orders, o => o.Tags, null,
        IndexDefinition.Create("Discount"), IndexDefinition.Create("Retail"), IndexDefinition.Create("New"), IndexDefinition.Create("Season"), IndexDefinition.Create("BestSeller")));

var tagYear = cubeTagYear.BreakdownByDimensions(0)
    .Select(row => new {
        Year = row.GetBoundIndexDefinition(0).Title ?? row.GetBoundIndexDefinition(0).Value,
        Discount = row.GetValue("Discount"),
        Retail = row.GetValue("Retail"),
        New = row.GetValue("New"),
        Season = row.GetValue("Season"),
        BestSeller = row.GetValue("BestSeller")
    })
    .Select(order => order.ObjectToDictionary())
    .ToList();

tagYear.DisplayAsTable();

index,Year,Discount,Retail,New,Season,BestSeller
0,2007,52,64,42,42,32
1,2008,14,12,14,0,54
2,2009,45,30,80,30,75
3,Total,111,106,136,72,161


## <a id='toc1_6_'></a>[Three-dimensional report (EmployeeId x CustomerId x Year)](#toc0_)

In [32]:
var cube3d = orders.BuildCube(
    AggregationDefinition.CreateForCollection(orders, o => o.Quantity, (a,b)=>a+b, 0m),
    DimensionDefinition.CreateForCollection(
        orders,
        o => o.EmployeeId.ToString(),
        "EmployeeId",
        IndexDefinition.Create("1"),
        IndexDefinition.Create("2"),
        IndexDefinition.Create("3"))
        .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(
        orders,
        o => o.CustomerId,
        "CustomerId",
        IndexDefinition.Create("A"),
        IndexDefinition.Create("B"),
        IndexDefinition.Create("C"),
        IndexDefinition.Create("D"))
        .WithTrailingDefaultIndex("Total"),
    DimensionDefinition.CreateForCollection(
        orders,
        o => o.OrderDate.Year.ToString(),
        "Year",
        IndexDefinition.Create("2007"),
        IndexDefinition.Create("2008"),
        IndexDefinition.Create("2009"))
        .WithTrailingDefaultIndex("Total"));

var table3d = cube3d.ToTable([0, 1], [2]);

table3d.DisplayTable();

EmployeeId,CustomerId,2007,2008,2009,Total
1,A,0,0,0,0
1,B,20,0,0,20
1,C,0,14,20,34
1,D,0,0,0,0
1,Total,20,14,20,54
2,A,0,40,0,40
2,B,0,12,15,27
2,C,0,0,0,0
2,D,0,0,0,0
2,Total,0,52,15,67


## <a id='toc1_7_'></a>[Zero / simple aggregation examples](#toc0_)

In [ ]:
Array.Empty<string>()
    .BuildCube<string, string, string>(
        AggregationDefinition.Create((string x) => x, (a, b) => default!, "Hello world"))
    .GetValue()
    .Display();

Enumerable.Range(1,10)
    .BuildCube<int, int, int>(AggregationDefinition.Create((int i) => i, (a, b) => a + b, 0))
    .GetValue()
    .Display();

Enumerable.Range(0,10)
    .BuildCube<int, int, int[]>(
        AggregationDefinition.Create((int i) => new[] { i }, (a,b)=> a.Concat(b).ToArray(), []))
    .GetValue()
    .Display();

Hello world

55

[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

## <a id='toc1_8_'></a>[Modulus dimension examples](#toc0_)

In [34]:
var mod3Cube = Enumerable.Range(0,10).BuildCube(
    AggregationDefinition.Create(
        (int i) => new[] { i }, (a, b) => a.Concat(b).ToArray(), Array.Empty<int>()),
    DimensionDefinition.Create(
        (int i) => (int?)(i % 3),
        "Mod 3",
        Enumerable.Range(0, 3).Select(i => IndexDefinition.Create((int?)i)).ToArray()));

mod3Cube.GetValue().Display();
mod3Cube.GetValue(0).Display();
mod3Cube.GetValue(1).Display();
mod3Cube.GetValue(2).Display();


[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

[ 0, 3, 6, 9 ]

[ 1, 4, 7 ]

[ 2, 5, 8 ]

In [35]:

DimensionDefinition<int, int?> CreateModulusDimension(int m) =>
    DimensionDefinition.Create(
        (int i)=> (int?)(i % m),
        $"Mod {m}",
        Enumerable.Range(0,m).Select(i => IndexDefinition.Create((int?)i)).ToArray());

var mod3and5Cube = Enumerable.Range(0,20).BuildCube(
    AggregationDefinition.Create(
        (int i)=> new[] { i }, (a, b) => a.Concat(b).ToArray(), Array.Empty<int>()),
    CreateModulusDimension(3),
    CreateModulusDimension(5));

mod3and5Cube.GetValue(0).Display();
mod3and5Cube.GetValue(null, 0).Display();
mod3and5Cube.GetValue(0, 0).Display();
mod3and5Cube.ToTable([0, 1], []).DisplayTable();

[ 0, 3, 6, 9, 12, 15, 18 ]

[ 0, 5, 10, 15 ]

[ 0, 15 ]

Mod 3,Mod 5,value
0,0,"[ 0, 15 ]"
0,1,[ 6 ]
0,2,[ 12 ]
0,3,"[ 3, 18 ]"
0,4,[ 9 ]
1,0,[ 10 ]
1,1,"[ 1, 16 ]"
1,2,[ 7 ]
1,3,[ 13 ]
1,4,"[ 4, 19 ]"
